# 雄獅開團及訂單資料處理

### 首次匯入資料處理區

In [40]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.options.display.float_format = '{:,.0f}'.format
import numpy as np
import datetime

In [2]:
#團估價系統資料匯入from 沅佑/Rebo
names_q_connect = ['團號','團控公司','估價代號','最低成團人數','總團位數','開團日期','保證出團日期']
q_connect = pd.read_csv("lion_product/q_connect.csv", names=names_q_connect)
q_connect['團號公司'] = q_connect['團號'].str.strip()+" "+q_connect['團控公司'].str.strip()

names_q_price = ['估價代號','估價檔次A','估價檔次B','標準批售價','標準直售價','標準批售毛利率','標準直售毛利率',\
                 '預定批售價','預定直售價','預定批售毛利率','預定直售毛利率']
q_price = pd.read_csv("lion_product/q_price.csv", names=names_q_price)

names_q_main = ['估價代號','進度','估價名稱','標準團名','線別','地區','估價效期','需求單位','維護','天數','估價幣別','版型','團型',\
                '估價代號匯率幣別','估價代號匯率','領隊人數','導遊人數','工作人員人數','建檔單位','預估檔次','維護日期']
q_main = pd.read_csv("lion_product/q_main.csv", names=names_q_main,index_col=False)
q_cost = pd.read_csv("lion_product/q_cost_renew.csv", index_col=False)

#團體及訂單資料來自彥興
names_main = ['團號','團控公司','產品事業處','產品部門','線別','副線別','區域別','新細地區','團型','標準團名','天數','團行程起始日',\
          '團控狀況','航空公司','保證出團','包機團','包團單位','訂單總數','取消單總數','總人數','營業額','一度毛利','團調整差','團補貼']
main = pd.read_csv("lion_product/PROD_TP_MAIN.csv", names=names_main, parse_dates=['團行程起始日'])
main['有效訂單數'] = main['訂單總數'] - main['取消單總數']
main['營業成本'] = main['營業額'] - main['一度毛利'] 
main['團號公司'] = main['團號'].str.strip()+" "+main['團控公司'].str.strip()
main['星期幾出發'] = main['團行程起始日'].dt.dayofweek+1

main['毛利率'] = ((main['一度毛利'] / main['營業額'])*100)
freq_bins = np.arange(0, 21, 1)
group_names = ['[{0}-{1}%]'.format(x, x+1) for x in freq_bins[0:-1]]
main['毛利率分組'] = pd.cut(main['毛利率'],freq_bins,labels=group_names)
main['毛利率分組'] = np.where(main['毛利率']<0,'負毛利',main['毛利率分組'])
main['毛利率分組'] = np.where(main['毛利率']>max(freq_bins),str(max(freq_bins))+'%以上',main['毛利率分組'])
main['毛利率'] = main['毛利率'].map('{:,.2f}'.format) 

names_price = ['團號','團控公司','直批別','價格方式','房型','售價_大人','售價_小孩佔床','售價_小孩不佔床','售價_小孩加床','售價_嬰兒']
price = pd.read_csv("lion_product/PDM_TP_PROFILE.csv", names=names_price)
price['團號公司'] = price['團號'].str.strip()+" "+price['團控公司'].str.strip()
price['直批別'][price['直批別']=='D']="直客"
price['直批別'][price['直批別']=='W']="同業"
price['價格方式'] = np.where(price['價格方式']=="1售價","1團員",price['價格方式'])
price['價格方式'] = np.where(price['價格方式']=="6TKT湊票","6湊票",price['價格方式'])
price_pivot = price.pivot_table(index=['團號公司'],columns=['直批別','價格方式'])
price_pivot.columns = ["_".join((i,j,k)) for i,j,k in price_pivot.columns]
price_pivoted = pd.DataFrame(price_pivot.to_records())

names_cost = ['團號','團控公司','支單年度','支單號碼','支單序號','支出類別','供應商','供應商類別','供應商國家','供應商城市',\
              '人','天','單價','幣別代碼','申請時外幣匯率','申請成本(NTD)','付款時外幣匯率','換算付款金額','支付對象']
cost = pd.read_csv("lion_product/TRAS_COST.csv", names=names_cost)
cost['團號公司'] = cost['團號'].str.strip()+" "+cost['團控公司'].str.strip()
cost['申請成本(原幣別)'] = cost['人'] * cost['天'] * cost['單價']

#將估價匯率資訊代入團體成本檔以作成本匯率差計算
cost = pd.merge(cost, q_connect[['團號公司','估價代號']], on=["團號公司"], how="left")
cost = pd.merge(cost, q_main[['估價代號','估價代號匯率幣別','估價代號匯率']], on=["估價代號"], how="left")
cost['估價匯率'] = np.where(cost['幣別代碼']==cost['估價代號匯率幣別'],cost['估價代號匯率'],cost['申請時外幣匯率']).astype('float')
cost['估價成本(NTD)'] = np.where(cost['估價匯率']==cost['申請時外幣匯率'],cost['申請成本(NTD)'],cost['申請成本(原幣別)'] * cost['估價匯率'])
cost['成本匯率差'] = cost['申請成本(NTD)'] - cost['估價成本(NTD)']

cost_original = cost.pivot_table(index=['團號公司'],columns=['幣別代碼'],values=['申請成本(原幣別)'],aggfunc=np.sum,fill_value=0)
cost_original.columns = ["_".join((i,j)) for i,j in cost_original.columns]
cost_original = pd.DataFrame(cost_original.to_records())

cost_original_NTD = cost.pivot_table(index=['團號公司'],columns=['幣別代碼'],values=['申請成本(NTD)'],aggfunc=np.sum,fill_value=0)
cost_original_NTD.columns = ["_".join((i,j)) for i,j in cost_original_NTD.columns]
cost_original_NTD = pd.DataFrame(cost_original_NTD.to_records())

cost_NTD = cost.pivot_table(index=['團號公司'],columns=['支出類別'],values=['申請成本(NTD)','估價成本(NTD)','成本匯率差'],\
                            aggfunc=np.sum,fill_value=0,margins=True)
cost_NTD.columns = ["_".join((i,j)) for i,j in cost_NTD.columns]
cost_NTD = pd.DataFrame(cost_NTD.to_records())

cost['估價代號匯率幣別'] = np.where(cost['估價代號匯率幣別'].isnull,'NTD',cost['估價代號匯率幣別'])
cost['估價代號匯率'] = np.where(cost['估價代號匯率'].isnull,1,cost['估價代號匯率'])
cost_currency = cost[['團號公司','估價代號匯率幣別', '估價代號匯率']].drop_duplicates().reset_index() #移除重覆列

names_order = ['團號','團控公司','訂單年度','訂單編號','通路處','客戶身份別','網站來源','訂單狀況','訂單是否已取消','團型為自由行',\
              '訂單日與出團日差距天數','機票張數','大人人數','小孩人數','嬰兒人數','領隊人數','FOC人數','湊票人數','JOIN人數',\
              '脫隊人數','取消人數','總人數','通路營業額','通路一度毛利','通路總一度毛利','總團費','團費','特別行程','證照',\
               '手續費機場稅','產品部成本','Local團餐費','Local團自費活動節目費','稅額','同業退佣金額','折讓']
order = pd.read_csv("lion_product/ORDR_MAIN_BA2_PROD.csv", names=names_order, index_col=False)
order['團號公司'] = order['團號'].str.strip()+" "+order['團控公司'].str.strip()
order['訂單號碼'] = order['訂單年度'].apply(str)+"-"+order['訂單編號'].apply(str)
order['客戶身份別'] = order['客戶身份別'].apply(lambda x: x.strip())
order['直客同業'] = np.where(order['客戶身份別'].str.contains('同行'), "同業", "直客")

lcc_list = pd.read_csv("lion_product/lcc_list.csv")

#"稱呼"欄位應抓錯地方，何以有T青少年及nan
#缺狀態(是否取消)
names_order_detail = ['訂單年度','訂單編號','訂單序號','佔床','條件','加稅','總團費','團費','特別行程','證照','手續費',\
                      '產品部成本','Local團餐費','Local團自費活動節目費','GST稅額','同業退佣金額','是否脫隊','稱呼']
order_detail = pd.read_csv("lion_product/order_detail.csv", names = names_order_detail, index_col=False)
order_detail[['訂單年度','訂單編號','訂單序號']] = order_detail[['訂單年度','訂單編號','訂單序號']].astype('object')
order_detail['條件'] = np.where(order_detail['條件'].isnull(),'7JOIN',order_detail['條件'])
order_detail['稱呼'] = np.where(order_detail['稱呼'].isnull(),'A成人',order_detail['稱呼'])
order_detail['稱呼'] = np.where(order_detail['稱呼']=='T青少年','A成人',order_detail['稱呼'])
order_detail['佔床'] = np.where(order_detail['佔床'].isnull(),'無',order_detail['佔床'])
order_detail['佔床'] = np.where(order_detail['稱呼']=='A成人','無',order_detail['佔床'])
order_detail['佔床'] = np.where(order_detail['稱呼']=='I嬰兒','無',order_detail['佔床'])
order_detail['訂單號碼'] = order_detail['訂單年度'].apply(str)+"-"+order_detail['訂單編號'].apply(str)

price_1 = price[['團號公司','直批別','價格方式','房型','售價_大人']].rename(columns={'售價_大人':'定價'})
price_1['稱呼'] = 'A成人'
price_1['佔床'] = '無'
price_2 = price[['團號公司','直批別','價格方式','房型','售價_小孩佔床']].rename(columns={'售價_小孩佔床':'定價'})
price_2['稱呼'] = 'C兒童'
price_2['佔床'] = '佔床'
price_3 = price[['團號公司','直批別','價格方式','房型','售價_小孩不佔床']].rename(columns={'售價_小孩不佔床':'定價'})
price_3['稱呼'] = 'C兒童'
price_3['佔床'] = '不佔'
price_4 = price[['團號公司','直批別','價格方式','房型','售價_小孩加床']].rename(columns={'售價_小孩加床':'定價'})
price_4['稱呼'] = 'C兒童'
price_4['佔床'] = '加床'
price_5 = price[['團號公司','直批別','價格方式','房型','售價_嬰兒']].rename(columns={'售價_嬰兒':'定價'})
price_5['稱呼'] = 'I嬰兒'
price_5['佔床'] = '無'

price_new = pd.concat([price_1,price_2,price_3,price_4,price_5])
price_new['團號_直批_稱呼_條件_佔床'] = price_new['團號公司']+price_new['直批別']+price_new['稱呼']+price_new['價格方式']+price_new['佔床']

order_new = order[['團號公司','訂單號碼','直客同業','通路處']]

order_price = pd.merge(order_detail, order_new, on=['訂單號碼'], how="left")
order_price['團號_直批_稱呼_條件_佔床'] = order_price['團號公司']+order_price['直客同業']+order_price['稱呼']+order_price['條件']+order_price['佔床']
order_price = pd.merge(order_price, price_new[['團號_直批_稱呼_條件_佔床','定價']], on=['團號_直批_稱呼_條件_佔床'], how="left")

order_price['定價'] = np.where(order_price['定價']==0,order_price['團費'],order_price['定價'])
order_price['定價'] = np.where(order_price['定價'].isnull(),order_price['團費'],order_price['定價'])
order_price['定價'] = np.where(order_price['團費']==0,0,order_price['定價'])
order_price['團費定價差異'] = order_price['團費']-order_price['定價']
order_price['團費定價差異_正數'] = np.where(order_price['團費定價差異']>0,order_price['團費定價差異'],0)
order_price['團費定價差異_負數'] = np.where(order_price['團費定價差異']<0,order_price['團費定價差異'],0)
order_price.to_csv('lion_product/order_price.csv', index = False)

order_price_pivoted = order_price[['團號公司','直客同業','團費','定價','團費定價差異','團費定價差異_正數','團費定價差異_負數']]\
.pivot_table(index=['團號公司'],columns=['直客同業'],aggfunc=np.sum,margins=True)
order_price_pivoted.columns = ["_".join((i,j)) for i,j in order_price_pivoted.columns]
order_price_pivoted = pd.DataFrame(order_price_pivoted.to_records())

order_price_pivoted_1 = order_price[['團號公司','通路處','團費','定價','團費定價差異','團費定價差異_正數','團費定價差異_負數']]\
.pivot_table(index=['團號公司'],columns=['通路處'],aggfunc=np.sum,margins=True)
order_price_pivoted_1.columns = ["_".join((i,j)) for i,j in order_price_pivoted_1.columns]
order_price_pivoted_1= pd.DataFrame(order_price_pivoted_1.to_records())

In [44]:
#增加團體狀態
date_before = pd.Timestamp(2018, 6, 5, 0)
main["團體狀態"] = "尚未出團-無訂單(無保出)"
main["團體狀態"][(main['團行程起始日']<=date_before) & (main['總人數']>0) & (main["保證出團"]=="Y")] = "已出團(已保出)"
main["團體狀態"][(main['團行程起始日']<=date_before) & (main['總人數']>0) & (main["保證出團"]=="N")] = "已出團(無保出)"
main["團體狀態"][(main['團行程起始日']<=date_before) & (main['總人數']==0)] = "取消"
main["團體狀態"][(main['團行程起始日']>date_before) & (main['總人數']>0) & (main["保證出團"]=="Y")] = "尚未出團-有訂單(已保出)"
main["團體狀態"][(main['團行程起始日']>date_before) & (main['總人數']>0) & (main["保證出團"]=="N")] = "尚未出團-有訂單(無保出)"
main["團體狀態"][(main['團行程起始日']>date_before) & (main['總人數']==0) & (main["保證出團"]=="Y")] = "尚未出團-無訂單(已保出)"
main["團體狀態"][main['團控狀況']=="取消"] = "取消"

#增加產品分級
main['產品分級'] = '系列-其他'
main['產品分級'][main["標準團名"].str.contains("尊爵")] = "尊爵"
main['產品分級'][main["標準團名"].str.contains("旗艦")] = "旗艦"
main['產品分級'][main["標準團名"].str.contains("經典")] = "經典"
main['產品分級'][main["標準團名"].str.contains("特選")] = "特選"
main['產品分級'][main["標準團名"].str.contains("超值")] = "特選"
main['產品分級'][main["標準團名"].str.contains("趴|自由|機加酒")] = "自由行"
main['產品分級'][main["產品部門"]=="TW5團體自由行"] = "自由行"
main['產品分級'][main["產品事業處"]=="TW02產二"] = "主題"
main['產品分級'][main["副線別"].str.contains("輪")] = "郵輪"
main['產品分級'][main["標準團名"].str.contains("游輪|CR|麗星|遊輪|郵輪")] = "郵輪"
main['產品分級'][main["產品事業處"]=="TW03產三"] = "客製"
main['產品分級'][main["包團單位"]!="  "] = "客製-待確認(包團)" #待確認此規則是否合理

special_name = "客製|包團|參訪|醫|扶輪|大慶|遠傳|中信|彰基|屏東|訂車|訂房|接送|九和|世洋|樂音雅集|環宇|台南|長昕|歡樂美東假期|佳佳|天海|\
LOCALTOUR|愛馬仕|明志|樂透|公會|企銀|禾馨|安聯|克麗緹娜|EMBA|自組|喬富|創新|富可|新壽|經國|嘉康利|瑪古拉|銘傳|寶佳|協會|富邦|南山|人壽|\
李珊珊|雅筑|獅子會|元培|中產|議會|市府|新店|維他露|委員會|北大|榮耀|賀寶芙|工程|機車|圓山|尼克森|九龍灣|三越|三優|大山|大椏|大吉|中來|\
印刷|公會|高中|大學|小學|國中|保險|巨恆|米果|台佳|光頡|台郡|忠欣|岱宇|衣媚爾|科技|診所|醫院|東菱|哈迪思|貿易|五金|海佃|博士倫|得益|詠利|\
機械|晶華|酒堡|普達|景山|捷翰|統一|中學|匯豐|銀行|雲海|匯科|朝聖|大阪台北|大阪高雄|東京高雄|冠州|特行程|敦煌馬拉松|三立|大仲|太達|玄山|見本|\
NOJOIN|太達|百翔|國小|建設|農會|美加金|恩真|海格斯|晉欣|延平|小琉球民宿|艾迪科|百翔|永安雅|全航|和昇|富康|翔禾|輪胎|珮思|悠翔|工業|\
河內高爾夫|大佳|京王|東森|華偉|群英|豪豐|華碩|南亞|法師|展笙|明道|學生團|導遊|台電|家族遊|好友團|義演|IDA|三洋|三盛|營造|台新|會計|紘瑞|\
藝術坊|代表|氣密窗|捷安特|遠巧|協祥|泰金|實業|議員|立委|院長|國防|立法|尚華|清潔|VIP|藥廠|LAX紐澳|驗船|友善環境|日月光|世倢|吉尼|桂蘭|\
高餐|資誠|高工|工商|百福|長伶|美最時|珍愛飛|海洲|景文|管委會|陳福明|啟昇|縣政府|上順|克斯|宗瑋|友邦|安德|林娥|翔豐|碩軟|安侯|里長團|興富發|\
基因|參展|研習團|校友|成大|台大|政大|王母娘娘|科妍|雷科|友良|世欣|德成|國家地理雜誌|平達|保經|欣富山|眼科|鮮饌|耶路撒冷東京|投資|蓮安|騰達|\
廣翰|金中聯|萬宙|誠睿|福懋|惠光|電信|九如|三富|上海北海道|合成化學|NJ|電子|高億|電機|聖達|傑期|禾藤|亞森|今喜|鋼鐵|長竑|幼奇|工會|倡發|\
異萊|萬機|科大|寶來麗|蒔秋|昱奕|雄菱|耶路九州|華順|利可安|裝潢|南鎮|全家|玉鎮|福利|雷科|泰西|錸恩帕斯|華盟|達順|日產化學|昇弘|超捷|昇達|\
超捷|上揚|華王|御風|佳冬|奇景|佛光山|168T|象神灣|房屋|宜新|吉泰|馬泰|特行|客制|機電|校友"
main['產品分級'][main["標準團名"].str.contains(special_name)] = "客製"

special_name_equal = ["泰國","荷比法","美西1日","美西7日","美西8日","美西四天","荷蘭9天","山西深度","蒙古深度","北京深度","寧夏深度",\
                      "東北深度","九州五日","九州四日","泰國5日","泰國4日","泰國5天","泰國六日","泰國五日","泰國四日","泰國三日",\
                      "泰國二日","泰北6日","泰北5日","清邁四日","清邁5日","普吉5日","蘇梅五日","馬新7日","馬新6日","馬新5日","馬新6天",\
                      "馬新五日","新馬6日","新馬六天","新馬七日","新檳6日","檳城3日","檳城5D","沙巴5日","美西10日","美西12日",\
                      "美西13日","美國10日","休士頓8日","東歐11天","西葡13日","杜拜全覽B","峇里島5日","峇里島五日","新加坡4日",\
                      "新加坡5日","新加坡6日","新加坡7日","新加坡8日","新加坡三日","新加坡五日","新加坡八日","馬泰五日遊","泰國6日1",\
                      "義大利13天","直航南寧德天","九州福鹿7日","九州紫藤5日","日本中部六日","馬來西亞四日","馬來西亞六日",\
                      "馬來西亞5日","馬來西亞7日","馬來西亞5天","新加坡10日","藍夢峇里4日","王子墨西哥馬雅文化11日","土瑞義","英國冰島",\
                      "主打鄭州河南5日","東京3日","東京7日","東京8日","一百年阪阪島波單車(高雄)5日","泰國9日","泰國畢旅8日","蘇梅5日",\
                      "吉檳4日","吉檳蘭卡威","沙巴7日","沙巴古晉5日","沙巴吉隆坡6日","馬來西亞3日","馬來西亞三日","馬來西亞8日",\
                      "馬來西亞雲頂五日","馬來西亞檳城6日","馬新六日","馬新5日遊","馬新八日","新加坡五天","新加坡六天","新加坡七天",\
                      "新印五日之旅","新馬五日","新馬四日","新馬8日","新馬棕梠五日","熱情沙巴四日","檳城4D","檳城吉隆坡5日",\
                      "檳城怡保五日","峇里島七日","畢旅峇里島5日","雅加達5D"]
main['產品分級'][main["標準團名"].isin(special_name_equal)] = "客製-待確認(全團名)"
main['產品分級'][main["航空公司"] == "XX聯營學生票"] = "客製"

main['產品分級'][main["標準團名"].str.contains("促銷票|票團|機票|旅遊票|計畫票|計劃票")] = "票團"
main['產品分級'][main["團型"]=="2票團"] = "票團"
# main['產品分級'][main["團體狀態"]=="取消"] = "取消"

#增加團SIZE
main['團SIZE'] = '0人'
main['團SIZE'][(main['總人數'] > 0) & (main['總人數'] <= 15)] = "1-15人"
main['團SIZE'][(main['總人數'] > 15) & (main['總人數'] <= 20)] = "16-20人"
main['團SIZE'][(main['總人數'] > 20) & (main['總人數'] <= 25)] = "21-25人"
main['團SIZE'][(main['總人數'] > 25) & (main['總人數'] <= 30)] = "26-30人"
main['團SIZE'][(main['總人數'] > 30) & (main['總人數'] <= 35)] = "31-35人"
main['團SIZE'][(main['總人數'] > 35) & (main['總人數'] <= 40)] = "36-40人"
main['團SIZE'][(main['總人數'] > 40)] = "40人以上"
main['團SIZE'][main["團體狀態"]=="取消"] = "取消"

#增加航空公司類別
airline_code = main['航空公司'].str.slice(0,2)
airline_type = airline_code.apply(lambda x: x in lcc_list['IATAcode'].tolist())
main['航空公司類別'] = np.where(airline_type, "低成本航空", "傳統航空")
main['航空公司類別'][main['航空公司'].isnull()] = "無航空公司"
main['航空公司類別'][main['航空公司']=="!!servicefee"] = "無航空公司"
main['航空公司類別'][main['航空公司']=="3A噴射快船"] = "無航空公司"
main['航空公司類別'][main['航空公司']=="XX聯營學生票"] = "無航空公司"

#網站來源
order_netfrom = order.pivot_table('總人數',index=['團號公司'],columns=['網站來源'],aggfunc=sum,fill_value=0,margins=True)
order_netfrom.columns = [i.strip()+"人數" for i in order_netfrom.columns]
order_netfrom =pd.DataFrame(order_netfrom.to_records())

#客戶身份別
order_customertype = order.pivot_table(['總人數','通路營業額','通路總一度毛利'],index=['團號公司'],columns=['客戶身份別'],aggfunc=sum,fill_value=0,margins=True)
order_customertype.columns = ["_".join((i,j.strip())) for i,j in order_customertype.columns]
order_customertype =pd.DataFrame(order_customertype.to_records())

#通路事業處
order_path = order.pivot_table(['總人數','通路營業額','通路總一度毛利'],index=['團號公司'],columns=['通路處'],aggfunc=sum,fill_value=0,margins=True)
order_path.columns = ["_".join((i,j)) for i,j in order_path.columns]
order_path =pd.DataFrame(order_path.to_records())

#訂單日與出團日差距天數
order['訂單日與出團日差距天數_人數為零歸零'] = np.where(order['總人數']==0, 0, order['訂單日與出團日差距天數'])

#直批各屬性人數

order['直客同業'] = np.where(order['客戶身份別'].str.contains('同行'), "同業", "直客")
order_count = order.pivot_table(index=['團號公司'],columns=['直客同業'],values=['大人人數','小孩人數','嬰兒人數','領隊人數','FOC人數',\
                                                           '湊票人數','JOIN人數','脫隊人數','取消人數','總人數'],fill_value=0,aggfunc=np.sum)
order_count.columns = ["_".join((i,j)) for i,j in order_count.columns]
order_count =pd.DataFrame(order_count.to_records())

#量值合計
order_summary = order.groupby(['團號公司','團型為自由行']).sum()
order_summary.drop(['訂單年度','訂單編號','訂單日與出團日差距天數'], axis=1, inplace=True)
order_summary =pd.DataFrame(order_summary.to_records())
print ("開始合併資料")

#資料合併
main_price = pd.merge(main, price_pivoted, on=["團號公司"], how="left")
main_price_cost = pd.merge(main_price, cost_NTD, on=["團號公司"], how="left")
main_price_cost = pd.merge(main_price_cost, cost_original, on=["團號公司"], how="left")
main_price_cost = pd.merge(main_price_cost, cost_original_NTD, on=["團號公司"], how="left")
main_price_cost = pd.merge(main_price_cost, cost_currency, on=["團號公司"], how="left")
del main,price,price_pivoted,main_price,cost_NTD,cost_original,cost_original_NTD,cost_currency,cost,q_connect,q_main,q_cost,q_price
order_summary = pd.merge(order_summary, order_netfrom, on=["團號公司"], how="left")
order_summary = pd.merge(order_summary, order_customertype, on=["團號公司"], how="left")
order_summary = pd.merge(order_summary, order_path, on=["團號公司"], how="left")
order_summary = pd.merge(order_summary, order_count, on=["團號公司"], how="left")
main_price_cost_order = pd.merge(main_price_cost, order_summary, on=["團號公司"], how="left")
del order,order_summary,order_netfrom,order_customertype,order_count,main_price_cost
main_price_cost_order = pd.merge(main_price_cost_order, order_price_pivoted, on=["團號公司"], how="left")
main_price_cost_order = pd.merge(main_price_cost_order, order_price_pivoted_1, on=["團號公司"], how="left")
print ("合併完成")

#新增欄位
main_price_cost_order['產品分級'][(main_price_cost_order["團型為自由行"] == "Y") & (main_price_cost_order['總人數_y']>0)] = "自由行"
main_price_cost_order['FIT票包裝'] = "N"
main_price_cost_order['FIT票包裝'][main_price_cost_order["機票張數"] > 0] = "Y"
main_price_cost_order['團數'] = np.where(main_price_cost_order['團體狀態']=='取消', 0, 1).astype(int)
main_price_cost_order['出團年'] = main_price_cost_order['團行程起始日'].dt.year.astype(object)
main_price_cost_order['出團月'] = main_price_cost_order['團行程起始日'].dt.month.astype(object)
main_price_cost_order['出團日'] = main_price_cost_order['團行程起始日'].dt.day.astype(object)
main_price_cost_order['整團負毛利'] = main_price_cost_order['通路總一度毛利']<0

開始合併資料
合併完成


In [45]:
main_price_cost_order.to_csv('lion_product/main_price_cost_order_v20180605.csv', index = False)
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# 重新匯入總檔使用區

In [47]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import datetime
main_price_cost_order = pd.read_csv("lion_product/main_price_cost_order_v20180605.csv", parse_dates=['團行程起始日'])
names_q_connect = ['團號','團控公司','估價代號','最低成團人數','總團位數','開團日期','保證出團日期']
q_connect = pd.read_csv("lion_product/q_connect.csv", names=names_q_connect)
q_connect['團號公司'] = q_connect['團號'].str.strip()+" "+q_connect['團控公司'].str.strip()
q_connect.drop(['團號','團控公司'], axis=1, inplace=True)

first_start = pd.Timestamp(2017, 1, 1,0)
first_end = pd.Timestamp(2017, 12, 31,0)
second_start = pd.Timestamp(2018, 1, 1,0)
second_end = pd.Timestamp(2018, 12, 31,0)
A = (main_price_cost_order['團行程起始日']>=first_start)&(main_price_cost_order['團行程起始日']<=first_end)
B = (main_price_cost_order['團行程起始日']>=second_start)&(main_price_cost_order['團行程起始日']<=second_end)
data_period = main_price_cost_order[(A)|(B)]
data_period = pd.merge(data_period, q_connect, on=["團號公司"], how="left")
data_period['總團位數(成團)'] = np.where(data_period['團體狀態']=="取消", 0, data_period['總團位數'])

data_period.to_csv('lion_product/main_price_cost_order_year17-18.csv', index = False)

# 估價成本檔資料錯位處理

In [39]:
#團估價成本檔，由於原始資料分隔號"逗號＂在內容有判讀問題，故需先處理
q_cost = pd.read_csv("lion_product/q_cost.csv", index_col=False, sep='？', encoding='utf8', names=['original_content'])
q_cost['count'] = q_cost['original_content'].str.count(',')

names_q_cost = ['估價代號','成本檔次','FOC(MAX)','成本大類','成本中類','名稱','標的物','供應商名稱','供應商名稱2',\
                '幣別','匯率','成本(Original)','單間房差']
equal12 = q_cost[q_cost['count'] == 12]
q_cost_equal12 = pd.DataFrame(equal12['original_content'].str.split(',').tolist(),columns = names_q_cost)
q_cost_equal12.drop(['供應商名稱2'], axis=1, inplace=True)

names_q_cost = ['估價代號','成本檔次','FOC(MAX)','成本大類','成本中類','名稱','標的物','供應商名稱','供應商名稱2','供應商名稱3',\
                '幣別','匯率','成本(Original)','單間房差']
equal13 = q_cost[q_cost['count'] == 13]
q_cost_equal13 = pd.DataFrame(equal13['original_content'].str.split(',').tolist(),columns = names_q_cost)
q_cost_equal13.drop(['供應商名稱2','供應商名稱3'], axis=1, inplace=True)

names_q_cost = ['估價代號','成本檔次','FOC(MAX)','成本大類','成本中類','名稱','標的物','供應商名稱','供應商名稱2','供應商名稱3',\
                '供應商名稱4','幣別','匯率','成本(Original)','單間房差']
equal14 = q_cost[q_cost['count'] == 14]
q_cost_equal14 = pd.DataFrame(equal14['original_content'].str.split(',').tolist(),columns = names_q_cost)
q_cost_equal14.drop(['供應商名稱2','供應商名稱3','供應商名稱4'], axis=1, inplace=True)

names_q_cost = ['估價代號','成本檔次','FOC(MAX)','成本大類','成本中類','名稱','標的物','供應商名稱','供應商名稱2','供應商名稱3',\
                '供應商名稱4','供應商名稱5','供應商名稱6','幣別','匯率','成本(Original)','單間房差']
equal16 = q_cost[q_cost['count'] == 16]
q_cost_equal16 = pd.DataFrame(equal16['original_content'].str.split(',').tolist(),columns = names_q_cost)
q_cost_equal16.drop(['供應商名稱2','供應商名稱3','供應商名稱4','供應商名稱5','供應商名稱6'], axis=1, inplace=True)

q_cost_renew = pd.concat([q_cost_equal12,q_cost_equal13,q_cost_equal14,q_cost_equal16]) 
q_cost_renew.to_csv('lion_product/q_cost_renew.csv', index = False)

# 成本分析

In [79]:
names_main = ['團號','團控公司','產品事業處','產品部門','線別','副線別','區域別','新細地區','團型','標準團名','天數','團行程起始日',\
          '團控狀況','航空公司','保證出團','包機團','包團單位','訂單總數','取消單總數','總人數','營業額','一度毛利','團調整差','團補貼']
main = pd.read_csv("lion_product/PROD_TP_MAIN.csv", names=names_main, parse_dates=['團行程起始日'])
main['團號公司'] = main['團號'].str.strip()+" "+main['團控公司'].str.strip()
main = main[['團號公司','團行程起始日']]

names_q_connect = ['團號','團控公司','估價代號','最低成團人數','總團位數']
q_connect = pd.read_csv("lion_product/q_connect.csv", names=names_q_connect)
q_connect['團號公司'] = q_connect['團號'].str.strip()+" "+q_connect['團控公司'].str.strip()
# q_connect = q_connect[q_connect['估價代號'].notnull()]
names_q_main = ['估價代號','進度','估價名稱','標準團名','線別','地區','估價效期','需求單位','維護','天數','估價幣別','版型','團型',\
                '估價代號匯率幣別','估價代號匯率','領隊人數','導遊人數','工作人員人數','建檔單位','預估檔次','維護日期']
q_main = pd.read_csv("lion_product/q_main.csv", names=names_q_main,index_col=False)
q_exrate = pd.merge(q_connect[['團號公司','估價代號']], q_main[['估價代號','估價代號匯率幣別','估價代號匯率']], on=["估價代號"], how="left")

names_cost = ['團號','團控公司','支單年度','支單號碼','支單序號','支出類別','供應商','供應商類別','供應商國家','供應商城市',\
              '人','天','單價','幣別代碼','申請時外幣匯率','申請成本(NTD)','付款時外幣匯率','換算付款金額','支付對象']
cost = pd.read_csv("lion_product/TRAS_COST.csv", names=names_cost)
cost['團號公司'] = cost['團號'].str.strip()+" "+cost['團控公司'].str.strip()
cost['申請成本(原幣別)'] = cost['人'] * cost['天'] * cost['單價']

cost = pd.merge(cost, main, on=["團號公司"], how="left")
cost = pd.merge(cost, q_exrate, on=["團號公司"], how="left")

cost['估價匯率'] = np.where(cost['幣別代碼']==cost['估價代號匯率幣別'],\
                                cost['估價代號匯率'],cost['申請時外幣匯率']).astype('float')
cost['估價成本(NTD)'] = np.where(cost['估價匯率']==cost['申請時外幣匯率'],\
                                     cost['申請成本(NTD)'],cost['申請成本(原幣別)'] * cost['估價匯率'])
cost['成本匯率差'] = cost['申請成本(NTD)'] - cost['估價成本(NTD)']

cost_combine = cost[['團號公司','團行程起始日','估價代號','支出類別','供應商','支付對象','幣別代碼','申請成本(原幣別)','申請成本(NTD)',\
                     '估價代號匯率幣別','估價成本(NTD)','成本匯率差']]\
.groupby(['團號公司','團行程起始日','支出類別','供應商','支付對象','估價代號','估價代號匯率幣別','幣別代碼']).sum()
cost_combine = pd.DataFrame(cost_combine.to_records())
cost_combine

,團號公司,團行程起始日,支出類別,供應商,支付對象,估價代號,估價代號匯率幣別,幣別代碼,申請成本(原幣別),申請成本(NTD),估價成本(NTD),成本匯率差
0,16AB929BR T,2016-09-29,11團費,AG201621_雄獅 LAX-LION,雄獅 LAX-LION,1AAW11-2016000055,USD,USD,10087.57,318211.0,340959.866,-22748.866
1,16AB929BR T,2016-09-29,11團費,TL_領隊,16AB929BR (T/L 徐祖蔭),1AAW11-2016000055,USD,NTD,150.00,150.0,150.000,0.000
2,16AB929BR T,2016-09-29,11團費,TL_領隊,16AB929BR (T/L 徐祖蔭),1AAW11-2016000055,USD,USD,4243.15,133806.0,143418.470,-9612.470
3,16AB929BR T,2016-09-29,12餐費,TL_領隊,16AB929BR (T/L 徐祖蔭),1AAW11-2016000055,USD,NTD,9000.00,9000.0,9000.000,0.000
4,16AB929BR T,2016-09-29,13導遊,TL_領隊,16AB929BR (T/L 徐祖蔭),1AAW11-2016000055,USD,USD,1725.00,54251.0,58305.000,-4054.000
5,16AB929BR T,2016-09-29,14票證,TL_領隊,16AB929BR (T/L 徐祖蔭),1AAW11-2016000055,USD,USD,1276.80,40136.0,43155.840,-3019.840
6,16AB929BR T,2016-09-29,17車資,BSUSLAX001_AMERICAN VACATION,AMERICAN VACATION,1AAW11-2016000055,USD,USD,6465.00,203938.0,218517.000,-14579.000
7,16AB929BR T,2016-09-29,17車資,ZTTWTPE177_雄獅通運股份有限公司,雄獅通運股份有限公司,1AAW11-2016000055,USD,NTD,10200.00,10200.0,10200.000,0.000
8,16AB929BR T,2016-09-29,20機票退票款,BR_BR,BR,1AAW11-2016000055,USD,NTD,-21500.00,-21500.0,-21500.000,0.000
9,16AB929BR T,2016-09-29,22票國,AG201601_雄獅－高雄,雄獅高雄票務,1AAW11-2016000055,USD,NTD,2007.00,2007.0,2007.000,0.000


In [80]:
cost_combine.to_csv('lion_product/cost_combine-v20180522.csv', index = False)

In [82]:
first_start = datetime.date(2017, 4, 1)
first_end = datetime.date(2017, 4, 30)
second_start = datetime.date(2018, 4, 1)
second_end = datetime.date(2018, 4, 30)
A = (cost_combine['團行程起始日']>=first_start)&(cost_combine['團行程起始日']<=first_end)
B = (cost_combine['團行程起始日']>=second_start)&(cost_combine['團行程起始日']<=second_end)
data_period = cost_combine[(A)|(B)]

main_price_cost_order = pd.read_csv("lion_product/main_price_cost_order_year17-18_month04.csv", encoding='cp950')
data_period = pd.merge(data_period,main_price_cost_order[['團號公司','產品事業處','產品部門','線別','副線別','區域別','新細地區',\
                                              '團型','標準團名','天數','出團年','出團月','航空公司','團體狀態','產品分級']],\
                       on=["團號公司"], how="left")
data_period.to_csv('lion_product/cost_combine-v20180522_year17-18_month04.csv', index = False)

In [44]:
cost_grouped = cost[cost['支出類別']=="11團費"][['供應商','支出類別','申請成本(NTD)']].groupby(['供應商','支出類別']).sum().reset_index()
cost_grouped['申請成本(NTD)'] = cost_grouped['申請成本(NTD)'].astype('int')
cost_grouped.sort_values('申請成本(NTD)', ascending=False)

,供應商,支出類別,申請成本(NTD)
640,TL_領隊,11團費,1251146887
421,LCCHZRH001_KUONI,11團費,1218315587
1,AG00201740_雙獅聯合國際旅行社股份有限公司,11團費,962927887
625,OTTWTPE031_英商康年華,11團費,950502598
568,LCUKLON001_TUMLARE,11團費,688887822
293,LC00000141_SHIN LAI TOUR,11團費,542562844
24,AG201624_雄獅 SYD-LION,11團費,538330577
47,AGFT0009_雄獅 YVR-LION,11團費,424194481
26,AG201643_雄獅 BKK-LION,11團費,394731434
447,LCCNHKG020_G2 Travel Limited,11團費,334670567


# 6月毛利預估資料處理

In [3]:
#自SERP毛利預估系統導出，進行排除雙獅/雄保訂單及無營業額者
detail_201806 = pd.read_csv("lion_product/detail_201806.csv",encoding='cp950')
detail_201806['團號公司'] = detail_201806['團號'].str.strip()+" "+detail_201806['公司'].str.strip()
detail_201806 = detail_201806[(detail_201806['公司'] != "ZP") & (detail_201806['公司'] != "LB") & (detail_201806['總營業額']>0)]

#毛利預估系統產出之報表無團號對應之產品部門，導入SAS產品主檔資料進行比照
#需再補＂團型＂
main_names = ['團號','團控公司','產品事業處','產品部門']
main_201806 = pd.read_csv("lion_product/main_201806.csv",names=main_names)
main_201806['團號公司'] = main_201806['團號'].str.strip()+" "+main_201806['團控公司'].str.strip()
main_201806.drop(['團號','團控公司'], axis=1, inplace=True)

#毛利預估系統產出之報表無團號對應之估價代號，資料來源為IT自毛利預估系統中匯出
names_q_connect = ['團號','團控公司','估價代號','最低成團人數','總團位數']
q_connect = pd.read_csv("lion_product/q_connect.csv", names=names_q_connect)
q_connect['團號公司'] = q_connect['團號'].str.strip()+" "+q_connect['團控公司'].str.strip()
q_connect.drop(['團號','團控公司'], axis=1, inplace=True)

#上述三資料庫主行合併
detail_201806 = pd.merge(detail_201806, main_201806, on=["團號公司"], how="left")
detail_201806_connected = pd.merge(detail_201806, q_connect, on=["團號公司"], how="left")

#針對合併後資料進行相關衍生欄位判讀
detail_201806_connected['預估檔次A'] = detail_201806_connected['預估檔次'].str.split('+').str[0].astype('int')
detail_201806_connected['團數'] = 1
detail_201806_connected['總人數'] = detail_201806_connected['KK人數']+detail_201806_connected['HK人數']+detail_201806_connected['HL人數']
detail_201806_connected['實際人數是否大於預估檔次'] = np.where(detail_201806_connected['總人數'] > detail_201806_connected['預估檔次A'],\
                                                   "YES","NO")

#針對合併後資料依產品部門彙整產出數據
s = detail_201806_connected.pivot_table(index=['產品部門'],columns=['實際人數是否大於預估檔次'],values=['團數'],\
                                    aggfunc=np.sum,fill_value=0,margins=True)
s.columns = ["_實際人數>預估檔次_".join((i,j)) for i,j in s.columns]
s =pd.DataFrame(s.to_records())
ss = detail_201806_connected.pivot_table(index='產品部門',values='估價代號',\
                                         aggfunc=lambda x: len(x.dropna().unique()),margins=True)\
                                        .rename(columns={'估價代號':'估價代號數量'})
ss =pd.DataFrame(ss.to_records())
sss = pd.merge(s,ss,on=['產品部門'],how="left")
sss['估價代號數量倍數'] = (sss['團數_實際人數>預估檔次_All']/sss['估價代號數量']).map('{:,.1f}'.format)
sss.rename(columns = {'團數_實際人數>預估檔次_All':'總團數'}, inplace=True)
sss['實際人數>預估檔次團數比重'] = ((sss['團數_實際人數>預估檔次_YES'] / sss['總團數'])*100).map('{:,.1f}'.format)
sss

,產品部門,團數_實際人數>預估檔次_NO,團數_實際人數>預估檔次_YES,總團數,估價代號數量,估價代號數量倍數,實際人數>預估檔次團數比重
0,TW11F太平洋小島,49,56,105,7,15.0,53.3
1,TW11美洲不含小島,76,39,115,52,2.2,33.9
2,TW12大洋洲,34,40,74,30,2.5,54.1
3,TW13歐洲,218,61,279,118,2.4,21.9
4,TW14亞非,48,24,72,28,2.6,33.3
5,TW15A大陸,330,438,768,300,2.6,57.0
6,TW15B港澳珠圳,290,380,670,124,5.4,56.7
7,TW16A東北亞日本,497,331,828,245,3.4,40.0
8,TW16C東北亞韓國,188,237,425,120,3.5,55.8
9,TW17東南亞,300,388,688,200,3.4,56.4


In [42]:
detail_201806_connected.to_csv('lion_product/detail_201806_connected.csv', index = False)

In [14]:
len(detail_201806_connected[detail_201806_connected['產品部門'].isnull()])

0

In [89]:
#各估價代號之毛利率
summary = detail_201806_connected.groupby(['產品部門','估價代號'])['團數','總人數','總營業額','總毛利'].sum().astype('float')
summary = pd.DataFrame(summary.to_records())
summary['毛利率']=((summary['總毛利']/summary['總營業額'])*100).map('{:,.1f}%'.format)
summary.sort_values(['產品部門','總營業額'],ascending=[True,False])

,產品部門,估價代號,團數,總人數,總營業額,總毛利,毛利率
0,TW11F太平洋小島,1FGU01-2017008385,53,239,"6,490,667","3,792,357",58.4%
6,TW11F太平洋小島,1FGU11-2017017399,22,79,"2,988,809","1,302,633",43.6%
1,TW11F太平洋小島,1FGU01-2017015205,17,52,"1,483,580","920,680",62.1%
2,TW11F太平洋小島,1FGU11-2017017237,9,34,"715,104","130,304",18.2%
4,TW11F太平洋小島,1FGU11-2017017242,2,5,"145,670","39,670",27.2%
3,TW11F太平洋小島,1FGU11-2017017241,1,2,"61,600","10,000",16.2%
5,TW11F太平洋小島,1FGU11-2017017244,1,2,"54,002","13,202",24.4%
46,TW11美洲不含小島,1BAPZ1-2018001037,3,252,"9,732,248","1,920,248",19.7%
26,TW11美洲不含小島,1AAW11-2017017166,6,138,"9,209,190","988,149",10.7%
42,TW11美洲不含小島,1BAC01-2017017994,6,100,"8,509,480","2,994,883",35.2%


In [5]:
detail_201806_connected[(detail_201806_connected['估價代號'].isnull()) & (detail_201806_connected['總營業額']>0)]

,團控公司,線別,地區,區域別,團號,公司,標準團名,天數,RC,月份,...,團號公司,產品事業處,產品部門,估價代號,最低成團人數,總團位數,預估檔次A,團數,總人數,實際人數是否大於預估檔次
159,台北,1 C 美東,AEN1 紐約,1C04 美東,18AR611BR1,T,特選瘋紐約8日,8,謝祐華,6,...,18AR611BR1 T,TW01產一,TW11美洲不含小島,NaN,nan,nan,41,1,2,NO
167,台北,1 B 加拿大,APZ1 多倫多,1B03 加東,18AP618BRK,T,臺中市政府加拿大2018ICLE參訪8日,8,余惠文,6,...,18AP618BRK T,TW01產一,TW11美洲不含小島,NaN,nan,nan,20,1,6,NO
202,台北,1 A 美國,AWL1 洛杉磯,1A01 美國,18AB620CIE,T,永達MDRT洛杉磯10日,10,石永璇,6,...,18AB620CIE T,TW03產三,TW31會獎產品,NaN,0,31,31,1,25,NO
237,台北,1 A 美國,AW11 美西,1A01 美國,18AB606BRK,T,高市府波特蘭參訪團8日,8,余惠文,6,...,18AB606BRK T,TW01產一,TW11美洲不含小島,NaN,nan,nan,15,1,14,NO
313,台北,2 A 澳洲,AU02 雙城,2A01 東澳交叉點,18PI624CIK,T,一口價經典極限下殺WIFI強打雪墨雙城8日,8,楊淑卿,6,...,18PI624CIK T,TW01產一,TW12大洋洲,NaN,nan,nan,20,1,2,NO
354,台北,3 A 中西歐,EU12 英法,3A02 多國中西歐,18EU629BRC,T,秒購特選英法雙國(雙點進出)10日,10,朱虹臻,6,...,18EU629BRC T,TW01產一,TW13歐洲,NaN,nan,nan,31,1,11,NO
382,台北,3 A 中西歐,EU10 德奧,3A02 多國中西歐,18EU618CIZ,T,德奧8日,8,陳奕岑,6,...,18EU618CIZ T,TW01產一,TW13歐洲,NaN,0,7,7,1,7,NO
701,台北,4 B 南亞,IN01 印度,4B02 南亞,18XI612CIK,T,旅展特選印度直飛8日,8,徐偉軒,6,...,18XI612CIK T,TW01產一,TW14亞非,NaN,nan,nan,25,1,6,NO
740,台中,5 B 港澳珠圳,HKM1 澳自由行,5B09 澳門,18CM616BR1,X,端午澳門金沙集團X3日,3,尤翌錡,6,...,18CM616BR1 X,TW01產一,TW15B港澳珠圳,NaN,nan,nan,2,1,2,NO
760,台中,5 B 港澳珠圳,HKM1 澳自由行,5B09 澳門,18CM606B7L,X,123長榮平日澳門X4日,4,尤翌錡,6,...,18CM606B7L X,TW01產一,TW15B港澳珠圳,NaN,nan,nan,2,1,2,NO


# 參考區

#groupby功能

pg = pdname.groupby('分群的欄位名稱')
pg.size() -->看各群的筆數
pg.get_group('TW11美洲不含小島')  -->查看特定族群的資料
pg.sum()

資料清理參考網站 https://kknews.cc/zh-tw/tech/knkevr.html

In [61]:
import pandas as pd
import numpy as np
parse_dates = ['049_團行程起始日']
to_num = {'金額_訂單國家幣別':np.float64, '金額_原始支單幣別': np.float64}
gd_cost = pd.read_csv("lion_product/cost_2018.csv", encoding="cp950", parse_dates=parse_dates, dtype=to_num)
gd_cost.columns = ['團行程起始日','標準團名','團控公司所在國家','產品事業處','產品部門','團號','團控公司','線別','副線別',\
                   '支出單號年度','支出單流水號','支出款項類別','供應商','供應商所在國家','幣別','人','天','單價','申請時外幣匯率',\
                   '訂單年度','訂單編號','金額_原始支單幣別','金額_訂單國家幣別']
gd_cost['金額_訂單國家幣別'].sum()

#航空只留第一個用於判讀起飛機場
gd_airline_only1route = gd_airline[gd_airline["航班序號"]==1]

#合併資料(增加估價代號，團位，最低成團人數)
gd_main_price_airline = pd.merge(gd_main_price_airline, gd_quotation, how="left", on=["團號", "團控公司"])

gd_main_price_airline_for_merge = gd_main_price_airline.copy()

aggr = {'營業額': 'sum','總一度毛利':'sum'}
t = gd_order_main[gd_order_main['團行程起始日']<="2018-03-31"].groupby(['產品部門','保證出團','出團年']).agg(aggr)
t['毛利率'] = t['總一度毛利']/t['營業額']*100
t['毛利率']

#查閱所有欄位名稱
for i in main_price_cost_order.columns:
    print (i)
    
%timeit pd.merge(main_price_cost, order_summary, on=["團號公司"], how="left")

,團行程起始日,標準團名,團控公司所在國家,產品事業處,產品部門,團號,團控公司,線別,副線別,支出單號年度,...,供應商所在國家,幣別,人,天,單價,申請時外幣匯率,訂單年度,訂單編號,金額_原始支單幣別,金額_訂單國家幣別
0,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,1,1,320.00,1.0000,2017.0,3025473.0,320.00,320.0
1,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,1,1,"2,160.00",1.0000,2017.0,3025473.0,2160.00,2160.0
2,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,1,1,160.00,1.0000,2017.0,3232724.0,160.00,160.0
3,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,1,1,160.00,1.0000,2017.0,3406328.0,160.00,160.0
4,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,24,1,"16,300.00",1.0000,NaN,NaN,391200.00,391200.0
5,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,25,1,"5,682.00",1.0000,NaN,NaN,142050.00,142050.0
6,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,1,1,487.00,1.0000,NaN,NaN,487.00,487.0
7,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,24,1,"3,734.00",1.0000,NaN,NaN,89616.00,89616.0
8,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,1,1,"1,520.00",1.0000,2017.0,3025473.0,1520.00,1520.0
9,2018-01-01,WIFI特選強打黃金雪梨雙城8日,中華民國,TW01產一,TW12大洋洲,18PI101CIB,T,2大洋洲,2A澳洲,2017,...,中華民國,NTD,1,1,"1,893.00",1.0000,2017.0,3232724.0,1893.00,1893.0


In [13]:
names_order = ['團號','團控公司','訂單年度','訂單編號','通路處','客戶身份別','網站來源','訂單狀況','訂單是否已取消','團型為自由行',\
              '訂單日與出團日差距天數','機票張數','大人人數','小孩人數','嬰兒人數','領隊人數','FOC人數','湊票人數','JOIN人數',\
              '脫隊人數','取消人數','總人數','通路營業額','通路一度毛利','通路總一度毛利','總團費','團費','特別行程','證照',\
               '手續費機場稅','產品部成本','Local團餐費','Local團自費活動節目費','稅額','同業退佣金額','折讓']
order = pd.read_csv("lion_product/ORDR_MAIN_BA2_PROD.csv", names=names_order, index_col=False)
order['團號公司'] = order['團號'].str.strip()+" "+order['團控公司'].str.strip()
order['訂單號碼'] = order['訂單年度'].apply(str)+"-"+order['訂單編號'].apply(str)
order['客戶身份別'] = order['客戶身份別'].apply(lambda x: x.strip())
order['直客同業'] = np.where(order['客戶身份別'].str.contains('同行'), "同業", "直客")

names_price = ['團號','團控公司','直批別','價格方式','房型','售價_大人','售價_小孩佔床','售價_小孩不佔床','售價_小孩加床','售價_嬰兒']
price = pd.read_csv("lion_product/PDM_TP_PROFILE.csv", names=names_price)
price['團號公司'] = price['團號'].str.strip()+" "+price['團控公司'].str.strip()
price['直批別'][price['直批別']=='D']="直客"
price['直批別'][price['直批別']=='W']="同業"
price['價格方式'] = np.where(price['價格方式']=="1售價","1團員",price['價格方式'])
price['價格方式'] = np.where(price['價格方式']=="6TKT湊票","6湊票",price['價格方式'])

##########################
#"稱呼"欄位應抓錯地方，何以有T青少年及nan
names_order_detail = ['訂單年度','訂單編號','訂單序號','佔床','條件','加稅','總團費','團費','特別行程','證照','手續費',\
                      '產品部成本','Local團餐費','Local團自費活動節目費','GST稅額','同業退佣金額','是否脫隊','稱呼']
order_detail = pd.read_csv("lion_product/order_detail.csv", names = names_order_detail, index_col=False)
order_detail[['訂單年度','訂單編號','訂單序號']] = order_detail[['訂單年度','訂單編號','訂單序號']].astype('object')
order_detail['條件'] = np.where(order_detail['條件'].isnull(),'7JOIN',order_detail['條件'])
order_detail['稱呼'] = np.where(order_detail['稱呼'].isnull(),'A成人',order_detail['稱呼'])
order_detail['稱呼'] = np.where(order_detail['稱呼']=='T青少年','A成人',order_detail['稱呼'])
order_detail['佔床'] = np.where(order_detail['佔床'].isnull(),'無',order_detail['佔床'])
order_detail['佔床'] = np.where(order_detail['稱呼']=='A成人','無',order_detail['佔床'])
order_detail['佔床'] = np.where(order_detail['稱呼']=='I嬰兒','無',order_detail['佔床'])
order_detail['訂單號碼'] = order_detail['訂單年度'].apply(str)+"-"+order_detail['訂單編號'].apply(str)

price_1 = price[['團號公司','直批別','價格方式','房型','售價_大人']].rename(columns={'售價_大人':'定價'})
price_1['稱呼'] = 'A成人'
price_1['佔床'] = '無'
price_2 = price[['團號公司','直批別','價格方式','房型','售價_小孩佔床']].rename(columns={'售價_小孩佔床':'定價'})
price_2['稱呼'] = 'C兒童'
price_2['佔床'] = '佔床'
price_3 = price[['團號公司','直批別','價格方式','房型','售價_小孩不佔床']].rename(columns={'售價_小孩不佔床':'定價'})
price_3['稱呼'] = 'C兒童'
price_3['佔床'] = '不佔'
price_4 = price[['團號公司','直批別','價格方式','房型','售價_小孩加床']].rename(columns={'售價_小孩加床':'定價'})
price_4['稱呼'] = 'C兒童'
price_4['佔床'] = '加床'
price_5 = price[['團號公司','直批別','價格方式','房型','售價_嬰兒']].rename(columns={'售價_嬰兒':'定價'})
price_5['稱呼'] = 'I嬰兒'
price_5['佔床'] = '無'

price_new = pd.concat([price_1,price_2,price_3,price_4,price_5])
price_new['團號_直批_稱呼_條件_佔床'] = price_new['團號公司']+price_new['直批別']+price_new['稱呼']+price_new['價格方式']+price_new['佔床']

order_new = order[['團號公司','訂單號碼','直客同業']]

order_price = pd.merge(order_detail, order_new, on=['訂單號碼'], how="left")
order_price['團號_直批_稱呼_條件_佔床'] = order_price['團號公司']+order_price['直客同業']+order_price['稱呼']+order_price['條件']+order_price['佔床']
order_price = pd.merge(order_price, price_new[['團號_直批_稱呼_條件_佔床','定價']], on=['團號_直批_稱呼_條件_佔床'], how="left")

In [15]:
order_price['定價'] = np.where(order_price['定價'].isnull(),order_price['團費'],order_price['定價'])
order_price['定價'] = np.where(order_price['團費']==0,0,order_price['定價'])
order_price_pivoted = order_price[['團號公司','團費','定價']].pivot_table(index=['團號公司'],aggfunc=np.sum)
order_price_pivoted.columns = ["_".join((i,j)) for i,j in order_price_pivoted.columns]
order_price_pivoted = pd.DataFrame(order_price_pivoted.to_records())
order_price_pivoted

In [44]:
price_new['價格方式'].unique()

array(['1售價', '7JOIN', '6TKT湊票'], dtype=object)

In [22]:
order_price[(order_price['條件']=='3FOC') & (order_price['定價'].isnull())]

,訂單年度,訂單編號,訂單序號,佔床,條件,加稅,總團費,團費,特別行程,證照,...,Local團自費活動節目費,GST稅額,同業退佣金額,是否脫隊,稱呼,訂單號碼,團號公司,直客同業,團號_直批_稱呼_條件_佔床,定價
101760,2016,7039,14,無,3FOC,N,"1,957",0,0,0,...,0,0,0,N,A成人,2016-7039,16CS110SCT T,同業,16CS110SCT T同業A成人3FOC無,nan
101761,2017,10761,5,無,3FOC,N,"1,695",0,0,0,...,0,0,0,N,A成人,2017-10761,17CZ505CZT T,同業,17CZ505CZT T同業A成人3FOC無,nan
101762,2016,20592,9,無,3FOC,N,"1,439",0,0,0,...,0,0,0,N,A成人,2016-20592,16CY429CIT T,同業,16CY429CIT T同業A成人3FOC無,nan
101763,2018,11480,18,無,3FOC,N,0,0,0,0,...,0,0,0,N,A成人,2018-11480,18CV318BR2 K,同業,18CV318BR2 K同業A成人3FOC無,nan
101764,2017,18812,8,無,3FOC,N,"2,129",0,0,0,...,0,0,0,N,A成人,2017-18812,17JK120BX9 K,同業,17JK120BX9 K同業A成人3FOC無,nan
101765,2016,20592,41,無,3FOC,N,"1,439",0,0,0,...,0,0,0,N,A成人,2016-20592,16CY429CIT T,同業,16CY429CIT T同業A成人3FOC無,nan
101766,2016,20592,44,無,3FOC,N,"1,439",0,0,0,...,0,0,0,N,A成人,2016-20592,16CY429CIT T,同業,16CY429CIT T同業A成人3FOC無,nan
101767,2018,22388,24,無,3FOC,N,0,0,0,0,...,0,0,0,N,A成人,2018-22388,18XM424TKA T,同業,18XM424TKA T同業A成人3FOC無,nan
101768,2016,29646,1,無,3FOC,N,0,0,0,0,...,0,0,0,N,A成人,2016-29646,16JS207BR1 T,直客,16JS207BR1 T直客A成人3FOC無,nan
101769,2016,29646,2,無,3FOC,N,0,0,0,0,...,0,0,0,N,A成人,2016-29646,16JS207BR1 T,直客,16JS207BR1 T直客A成人3FOC無,nan


In [17]:
order_price['定價'] = np.where((order_price['團費']==0) & (order_price['條件']!="3FOC"),0,order_price['定價'])
order_price

,訂單年度,訂單編號,訂單序號,佔床,條件,加稅,總團費,團費,特別行程,證照,...,Local團自費活動節目費,GST稅額,同業退佣金額,是否脫隊,稱呼,訂單號碼,團號公司,直客同業,團號_直批_稱呼_條件_佔床,定價
0,2018,5696,1,無,1團員,N,"19,000","17,000","2,000",0,...,0,0,0,N,A成人,2018-5696,18CW203BRX K,同業,18CW203BRX K同業A成人1團員無,"17,388"
1,2018,5696,2,無,1團員,N,"19,000","17,000","2,000",0,...,0,0,0,N,A成人,2018-5696,18CW203BRX K,同業,18CW203BRX K同業A成人1團員無,"17,388"
2,2018,5696,4,無,1團員,N,"19,000","17,000","2,000",0,...,0,0,0,N,A成人,2018-5696,18CW203BRX K,同業,18CW203BRX K同業A成人1團員無,"17,388"
3,2015,11321,1,無,1團員,N,"76,888","76,888",0,0,...,0,0,0,N,A成人,2015-11321,15AN122CI T,直客,15AN122CI T直客A成人1團員無,"77,888"
4,2017,1535,1,無,1團員,N,0,0,0,0,...,0,0,0,N,A成人,2017-1535,17JK221BR T,直客,17JK221BR T直客A成人1團員無,0
5,2017,1840,1,無,1團員,N,"18,699","18,699",0,0,...,0,0,0,N,A成人,2017-1840,17SM202CIV T,同業,17SM202CIV T同業A成人1團員無,"18,999"
6,2017,1840,3,無,1團員,N,"18,699","18,699",0,0,...,0,0,0,N,A成人,2017-1840,17SM202CIV T,同業,17SM202CIV T同業A成人1團員無,"18,999"
7,2017,1840,4,無,1團員,N,"18,699","18,699",0,0,...,0,0,0,N,A成人,2017-1840,17SM202CIV T,同業,17SM202CIV T同業A成人1團員無,"18,999"
8,2016,2563,1,無,1團員,N,"29,400","29,400",0,0,...,0,0,0,N,A成人,2016-2563,16CA210BRA T,直客,16CA210BRA T直客A成人1團員無,"29,900"
9,2016,2563,2,無,1團員,N,"29,400","29,400",0,0,...,0,0,0,N,A成人,2016-2563,16CA210BRA T,直客,16CA210BRA T直客A成人1團員無,"29,900"


In [26]:
q_connect.head()

,估價代號,最低成團人數,總團位數,開團日期,保證出團日期,團號公司
0,-,4,20,NaN,NaN,18TI101SSX T
1,-,0,27,NaN,NaN,18TI113KCH T
2,-,0,17,2017-12-19 15:01:46.523,NaN,18TI113SUH T
3,-,0,55,NaN,NaN,18TI120CCC T
4,-,0,16,NaN,NaN,18TI217VIP T


In [28]:
data_period['估價代號']

0        1AAW11-2017006435
1        1AAW11-14ABD09BRC
2        1AAW11-2017004856
3                      NaN
4                      NaN
5        1AAW11-2016007111
6        1AAW11-2017000722
7                      NaN
8        1AAW11-2016003043
9                      NaN
10       1AAW11-2016003043
11                     NaN
12                     NaN
13                     NaN
14       1AAW11-2016006823
15                     NaN
16                     NaN
17                     NaN
18       1AAW11-2016003234
19       1AUS11-2017005146
20       1AAW11-2017004655
21       1AAW11-2017003829
22                     NaN
23                     NaN
24                     NaN
25                     NaN
26       1AAW11-2016005899
27                     NaN
28       1AAW11-2017007522
29       1AAW11-2016003043
               ...        
34272                  NaN
34273                  NaN
34274                  NaN
34275                  NaN
34276                  NaN
34277    6AJP21-2017021298
3

In [29]:
data_period.to_csv('lion_product/main_price_cost_order_year17-18_month04.csv', index = False)

In [30]:
data_period['估價代號']

0        1AAW11-2017006435
1        1AAW11-14ABD09BRC
2        1AAW11-2017004856
3                      NaN
4                      NaN
5        1AAW11-2016007111
6        1AAW11-2017000722
7                      NaN
8        1AAW11-2016003043
9                      NaN
10       1AAW11-2016003043
11                     NaN
12                     NaN
13                     NaN
14       1AAW11-2016006823
15                     NaN
16                     NaN
17                     NaN
18       1AAW11-2016003234
19       1AUS11-2017005146
20       1AAW11-2017004655
21       1AAW11-2017003829
22                     NaN
23                     NaN
24                     NaN
25                     NaN
26       1AAW11-2016005899
27                     NaN
28       1AAW11-2017007522
29       1AAW11-2016003043
               ...        
34272                  NaN
34273                  NaN
34274                  NaN
34275                  NaN
34276                  NaN
34277    6AJP21-2017021298
3